In [276]:
from pymongo import MongoClient
import pandas as pd

In [277]:
mongo_client = MongoClient('localhost', 27017)
mongo_db = mongo_client['inatel']
mongo_collection = mongo_db['in242']

In [302]:
dados = list(mongo_collection.find())
df = pd.DataFrame(dados)

In [279]:
df.head()

,_id,temperatura,data_coleta
0,5d98a7ebd65a5840a8f76098,23.267567,2019-10-05 11:25:47.754
1,5d98a7f0d65a5840a8f76099,24.573758,2019-10-05 11:25:52.764
2,5d98a7f5d65a5840a8f7609a,24.390460,2019-10-05 11:25:57.767
3,5d98a7fad65a5840a8f7609b,23.379668,2019-10-05 11:26:02.776
4,5d98a7ffd65a5840a8f7609c,24.666627,2019-10-05 11:26:07.799


In [281]:
df.count()

_id            3153
temperatura    3153
data_coleta    3153
dtype: int64

In [282]:
df['temperatura'].mean()

23.736967850684856

In [283]:
df['temperatura'].max()

29.997777694055955

In [284]:
df['temperatura'].min()

17.00540524657138

In [305]:
df = df.drop(columns=['_id'])

In [286]:
def check_temperatura(temperatura):
    if temperatura < 21:
        return 'Frio'
    if temperatura < 26:
        return 'Normal'
    return 'Quente'

In [287]:
check_temperatura(32.5)

'Quente'

In [288]:
df['sensacao'] = df['temperatura'].apply(check_temperatura)

In [289]:
df.tail()

,temperatura,data_coleta,sensacao
3148,24.465962,2019-10-05 15:48:24.871,Normal
3149,19.580395,2019-10-05 15:48:29.878,Frio
3150,17.065905,2019-10-05 15:48:34.883,Frio
3151,24.808047,2019-10-05 15:48:39.891,Normal
3152,21.638686,2019-10-05 15:48:44.896,Normal


In [290]:
df.groupby(['sensacao']).count()

,temperatura,data_coleta
sensacao,,
Frio,566,566
Normal,2010,2010
Quente,577,577


In [291]:
df.groupby(['sensacao'])['temperatura'].mean()

sensacao
Frio      19.098414
Normal    23.835397
Quente    27.944209
Name: temperatura, dtype: float64

In [292]:
media_min = df.set_index('data_coleta').resample('min')['temperatura'].mean()

In [293]:
media_min.head()

data_coleta
2019-10-05 11:25:00    24.077262
2019-10-05 11:26:00    23.724837
2019-10-05 11:27:00    24.063469
2019-10-05 11:28:00    24.150694
2019-10-05 11:29:00    23.697774
Freq: T, Name: temperatura, dtype: float64

In [294]:
[df['temperatura'].mean() - 2*df['temperatura'].std() ,df['temperatura'].mean() + 2*df['temperatura'].std() ]

[18.007307187848305, 29.466628513521407]

In [303]:
df.set_index('data_coleta', inplace=True)

In [307]:
df.head()

,temperatura
data_coleta,
2019-10-05 11:25:47.754,23.267567
2019-10-05 11:25:52.764,24.573758
2019-10-05 11:25:57.767,24.390460
2019-10-05 11:26:02.776,23.379668
2019-10-05 11:26:07.799,24.666627


In [308]:
df.reset_index(inplace=True)

In [309]:
df.tail()

,data_coleta,temperatura
3183,2019-10-05 15:51:20.122,23.522107
3184,2019-10-05 15:51:25.128,22.391761
3185,2019-10-05 15:51:30.134,22.742294
3186,2019-10-05 15:51:35.140,24.365541
3187,2019-10-05 15:51:40.155,20.233425


In [310]:
# Qual o minuto que tem a media de temperatura mais alta?
q1 = df.set_index('data_coleta').resample('min').mean()

In [311]:
q1.loc[q1.idxmax()['temperatura']]

temperatura    26.082001
Name: 2019-10-05 14:32:00, dtype: float64

In [313]:
# Sinalize todas a medicoes que ultrapassam 28 com alerta true. 

#def ver_alerta(temp):
#     if temp >= 28:
#         return True
#     return False
# df['Alerta'] = df['temperatura'].apply(ver_alerta)

df['Alerta'] = df['temperatura'].apply(lambda x: True if x >= 28 else False )

In [314]:
df

,data_coleta,temperatura,Alerta
0,2019-10-05 11:25:47.754,23.267567,False
1,2019-10-05 11:25:52.764,24.573758,False
2,2019-10-05 11:25:57.767,24.390460,False
3,2019-10-05 11:26:02.776,23.379668,False
4,2019-10-05 11:26:07.799,24.666627,False
...,...,...,...
3183,2019-10-05 15:51:20.122,23.522107,False
3184,2019-10-05 15:51:25.128,22.391761,False
3185,2019-10-05 15:51:30.134,22.742294,False
3186,2019-10-05 15:51:35.140,24.365541,False


In [315]:
df[df['Alerta'] == True]

,data_coleta,temperatura,Alerta
1288,2019-10-05 13:13:12.987,29.214825,True
1295,2019-10-05 13:13:48.029,28.707304,True
1298,2019-10-05 13:14:03.047,29.037690,True
1307,2019-10-05 13:14:48.121,28.522929,True
1308,2019-10-05 13:14:53.127,29.199075,True
...,...,...,...
3156,2019-10-05 15:49:04.927,28.394121,True
3162,2019-10-05 15:49:34.962,29.945071,True
3167,2019-10-05 15:50:00.005,28.270503,True
3168,2019-10-05 15:50:05.011,29.813120,True


In [316]:
# Gerar um DF consolidado com media, min e max e hora.
df2 = pd.DataFrame()

In [317]:
df2['media'] = df.set_index('data_coleta').resample('H')['temperatura'].mean()

In [318]:
df2

,media
data_coleta,
2019-10-05 11:00:00,23.971357
2019-10-05 12:00:00,23.987006
2019-10-05 13:00:00,23.773818
2019-10-05 14:00:00,23.506318
2019-10-05 15:00:00,23.491671


In [319]:
df2['minimo'] = df.set_index('data_coleta').resample('H')['temperatura'].min()

In [320]:
df2

,media,minimo
data_coleta,,
2019-10-05 11:00:00,23.971357,23.000088
2019-10-05 12:00:00,23.987006,23.005302
2019-10-05 13:00:00,23.773818,17.005691
2019-10-05 14:00:00,23.506318,17.005405
2019-10-05 15:00:00,23.491671,17.038183


In [321]:
df2['maximo'] = df.set_index('data_coleta').resample('H')['temperatura'].max()

In [322]:
df2

,media,minimo,maximo
data_coleta,,,
2019-10-05 11:00:00,23.971357,23.000088,24.999006
2019-10-05 12:00:00,23.987006,23.005302,24.998962
2019-10-05 13:00:00,23.773818,17.005691,29.997508
2019-10-05 14:00:00,23.506318,17.005405,29.997778
2019-10-05 15:00:00,23.491671,17.038183,29.978851


In [323]:
# outra possibilidade de resolucao
df2 = df.set_index('data_coleta').resample('H')['temperatura'].agg(['mean','min','max'])

In [324]:
df2

,mean,min,max
data_coleta,,,
2019-10-05 11:00:00,23.971357,23.000088,24.999006
2019-10-05 12:00:00,23.987006,23.005302,24.998962
2019-10-05 13:00:00,23.773818,17.005691,29.997508
2019-10-05 14:00:00,23.506318,17.005405,29.997778
2019-10-05 15:00:00,23.491671,17.038183,29.978851


In [238]:
# atribuir um valor NaN (Not a Number) em uma coluna
# df2['soma'] = pd.np.nan

In [325]:
# reordenar
df2 = df2[['max','min','mean']]

In [326]:
df2

,max,min,mean
data_coleta,,,
2019-10-05 11:00:00,24.999006,23.000088,23.971357
2019-10-05 12:00:00,24.998962,23.005302,23.987006
2019-10-05 13:00:00,29.997508,17.005691,23.773818
2019-10-05 14:00:00,29.997778,17.005405,23.506318
2019-10-05 15:00:00,29.978851,17.038183,23.491671


In [254]:
df2.to_excel('consolidado.xls')